# Download Library

In [1]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers[torch] -U
!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu
!pip install "langchain-core>=0.3.0,<0.4" --upgrade

In [2]:
!pip install peft

In [3]:
'''!pip install fastapi  # 실시간 챗봇 관련 인스톨 항목
!pip install uvicorn
!pip install transformers
!pip install torch
!pip install websockets'''

'!pip install fastapi  # 실시간 챗봇 관련 인스톨 항목\n!pip install uvicorn\n!pip install transformers\n!pip install torch\n!pip install websockets'

# Import Library

In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from peft import PeftModel

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
!docker run -d -p 6379:6379 redis # 캐싱

e9630fa4958ca4a928fb5d4f09a3db757f2ec9c075f644936830d79f9fd0cebd
docker: Error response from daemon: driver failed programming external connectivity on endpoint sad_mendeleev (234c75734ceb27fded49db6b9525907f5942928a83a171a6e97e66787b24877e): Bind for 0.0.0.0:6379 failed: port is already allocated.


In [6]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
# !pip install -r requirements.txt # 종속성 설치

In [8]:
pip install -U langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


# Vector DB

In [9]:
def process_pdf(file_path, chunk_size=256, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-base"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)

        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr",
                                    search_kwargs={'k': 3, 'fetch_k': 8})

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


# DB 생성

In [10]:
base_directory = "data/" # Your Base Directory
df = pd.read_csv(base_directory + 'test (3).csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|                                    | 0/6 [00:00<?, ?it/s]

Processing 보건복지부_노인장기요양보험 사업운영...


/tmp/ipykernel_12483/1818359745.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
Processing PDFs:  17%|████▋                       | 1/6 [00:04<00:24,  4.88s/it]

Processing 2024 장애인일자리 사업안내...


Processing PDFs:  33%|█████████▎                  | 2/6 [00:09<00:18,  4.68s/it]

...

Processing PDFs:  67%|██████████████████▋         | 4/6 [00:16<00:07,  3.94s/it]

Processing 서울시복지재단_서울시장애인가족지원센터...


Processing PDFs:  83%|███████████████████████▎    | 5/6 [00:21<00:04,  4.49s/it]

Processing 2024 장애인연금 사업안내...


Processing PDFs: 100%|████████████████████████████| 6/6 [00:27<00:00,  4.66s/it]


In [11]:
torch.cuda.empty_cache()

# MODEL Import

In [12]:
# def setup_llm_pipeline():
#     # 4비트 양자화 설정
#     bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.bfloat16
#     )

#     # 모델 ID
#     model_id = "rtzr/ko-gemma-2-9b-it"

#     # 토크나이저 로드 및 설정
#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     tokenizer.use_default_system_prompt = False

#     # 모델 로드 및 양자화 설정 적용
#     model = Gemma2ForCausalLM.from_pretrained(
#         model_id,
#         quantization_config=bnb_config,# 양자화 설정
#         device_map="auto",
#         trust_remote_code=True )

# #     model = PeftModel.from_pretrained(model, "./persona/checkpoint-200",is_trainable=True)

#     # HuggingFacePipeline 객체 생성
#     text_generation_pipeline = pipeline(
#         model=model,
#         tokenizer=tokenizer,
#         task="text-generation",
#         #temperature=0.2,
#         return_full_text=False,
#         max_new_tokens=450,
#     )

#     hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

#     return hf

In [13]:
# LLM 파이프라인
# llm = setup_llm_pipeline()

# Langchain 을 이용한 추론

In [14]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# # DataFrame의 각 행에 대해 처리
# for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
#     # 소스 문자열 정규화
#     source = normalize_string(row['Source'])
#     question = row['Question']

#     # 정규화된 키로 데이터베이스 검색
#     normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
#     retriever = normalized_keys[source]['retriever']

#     # RAG 체인 구성
#     template = """
#     당신은 복지 정보 전문가입니다. 아래의 정보를 바탕으로 질문에 대해 정확하고 신뢰할 수 있는 답변을 제공하세요:
    
#     {context}
    
#     질문: {question}
    
#     답변은 구체적이고 정확한 정보만 포함되도록 작성하세요. 가능하다면 관련 법률, 규정 또는 복지 서비스 이름을 명시하세요. 문장은 명확하고 간결하게 작성하세요.
    
#     답변:
#     """
#     prompt = PromptTemplate.from_template(template)

#     # RAG 체인 정의
#     rag_chain = (
#         {"context": retriever | format_docs, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#         | StrOutputParser()
#     )

#     # 답변 추론
#     print(f"Question: {question}")
#     full_response = rag_chain.invoke(question)
#     full_response = "\n".join([line.strip() for line in full_response.splitlines() if line.strip()])
#     print(f"Answer: {full_response}\n")

#     # 결과 저장
#     results.append({
#         "Source": row['Source'],
#         "Source_path": row['Source_path'],
#         "Question": question,
#         "Answer": full_response
#     })

In [15]:
!pip install fastapi uvicorn pydantic

In [16]:
!pip install --upgrade langchain-core langchain-huggingface

In [17]:
!pip install langchain-teddynote

In [18]:
import threading
import uvicorn

In [19]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# FastAPI 앱 생성 및 CORS 설정
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 질문을 담을 데이터 모델 정의
class QuestionRequest(BaseModel):
    question: str

# 루트 엔드포인트 확인
@app.get("/")
async def read_root():
    return {"message": "서버가 정상적으로 실행 중입니다!"}

# 모델을 한 번만 로드하는 함수
def setup_llm_pipeline():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model_id = "rtzr/ko-gemma-2-9b-it"

    # 토크나이저와 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    
    torch.cuda.empty_cache()
    
    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=250,
    )

    torch.cuda.empty_cache()


    return HuggingFacePipeline(pipeline=text_generation_pipeline)

# FastAPI 시작 시 LLM을 한 번만 로드
if not hasattr(app.state, "llm"):  # LLM이 아직 로드되지 않은 경우에만 로드
    app.state.llm = setup_llm_pipeline()

@app.post("/ask/")
async def ask_question(request: QuestionRequest):
    question = request.question
    print(f"사용자의 질문: {question}")

    # 데이터베이스 전체에서 질문에 맞는 문서 검색
    retrievers = [v['retriever'] for v in pdf_databases.values()]  # 모든 retriever 가져오기

    # 모든 retriever에서 관련 문서를 검색 (결과 합치기)
    all_docs = []
    for retriever in retrievers:
        docs = retriever.invoke(question)  # invoke로 문서 검색
        all_docs.extend(docs)

    # 검색된 문서들을 하나의 문자열로 포맷팅
    context = format_docs(all_docs)

    # RAG 체인 구성 (문맥과 질문을 처리)
    prompt_template = """
    당신은 복지 제도에 대해 깊은 이해를 가진 전문 상담사입니다. 아래의 정보와 법률, 규정에 기반하여 정확하고 신뢰할 수 있는 답변을 제공하세요:
    
    {context}
    
    질문: {question}
    
    답변은 다음과 같은 방식을 따르도록 작성하세요:
    1. 중요한 정보는 명확하고 간결하게 전달하세요.
    2. 답변에서 관련 법률, 조항, 또는 구체적인 복지 서비스 명칭을 포함하세요.
    3. 필요한 경우, 신청 절차나 서류에 대한 자세한 설명을 추가하세요.
    4. 답변을 구성할 때, 사용자에게 이해하기 쉬운 방식으로 설명하세요.
    5. 200자 이내로 답변을 구성하세요.
    
    답변:
    """
    prompt = PromptTemplate.from_template(prompt_template)

    # RAG 체인 정의
    rag_chain = (
        prompt
        | app.state.llm  # LLM을 사용한 추론
        | StrOutputParser()
    )

    # 답변 추론 수행
    full_response = rag_chain.invoke({"context": context, "question": question})
    full_response = "\n".join([line.strip() for line in full_response.splitlines() if line.strip()])

    torch.cuda.empty_cache()

    return {"question": question, "answer": full_response}


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_12483/2066715800.py:64: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=text_generation_pipeline)


In [20]:
# FastAPI 서버 실행 함수
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)  # reload=True 제거

# 스레드를 사용하여 백그라운드에서 서버 실행
thread = threading.Thread(target=run)
thread.start()


INFO:     Started server process [12483]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


사용자의 질문: 장애인 생계급여 기준에 대해서 알고싶어


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


INFO:     127.0.0.1:58498 - "POST /ask/ HTTP/1.1" 200 OK
사용자의 질문: 장애인 기준 정보 알려줘
INFO:     127.0.0.1:57350 - "POST /ask/ HTTP/1.1" 200 OK
사용자의 질문: 청각장애인이 받을 수 있는 복지 정보 하나 알려줘
INFO:     210.94.220.230:62364 - "POST /ask/ HTTP/1.1" 200 OK
사용자의 질문: 내가 30살이고, 앞이 안보이는 정도의 시각장애인이야. 내가 구청에 갈 일이 있어서 이동에 관련된 지원을 받을 수 있을까?
INFO:     210.94.220.230:62462 - "POST /ask/ HTTP/1.1" 200 OK
사용자의 질문: 나 갑자기 일하고 싶은데 일자리 어떻게 찾을 수 있을까?
INFO:     210.94.220.230:62659 - "POST /ask/ HTTP/1.1" 200 OK
사용자의 질문: 병원에서 정기적인 점검을 받아야 한다고 하는데 이런 지원도 받을 수 있을까?
INFO:     210.94.220.230:62741 - "POST /ask/ HTTP/1.1" 200 OK
사용자의 질문: 나 갑자기 살 자리가 없어졌어. 내가 장애등급이 있긴 한데 살 곳을 구할 수 있을까?
INFO:     210.94.220.230:62822 - "POST /ask/ HTTP/1.1" 200 OK


In [23]:
import requests
import json

# API 엔드포인트 URL
url = "http://localhost:8000/ask/"

# 보낼 데이터
data = {"question": "장애인 기준 정보 알려줘"}

# POST 요청 보내기
response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

# 응답 출력
print(response.json())


{'question': '장애인 기준 정보 알려줘', 'answer': "장애인 기준은 '장애인등록'을 통해 결정됩니다.  '장애인등록'은 장애인의 권익을 보호하고 지원을 받을 수 있도록 하는 중요한 절차입니다.\n장애인등록은 장애인의 종류와 정도에 따라 1~6등급으로 나뉘며, 각 등급별로 지원 대상이 다릅니다.\n자세한 내용은 보건복지부 홈페이지(https://www.mohw.go.kr/) 또는 지역 주민센터에 문의하세요."}


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 32145